## Python для работы с базами данных

Изменения в Vagrantfile:
1. Перед запуском виртуалки раскомментировать строку 50
```bash
config.vm.network "forwarded_port", guest: 5432, host: 5432, host_ip: "127.0.0.1"
```

2. На виртуалке меняем в конфигурационном файле postgresql метод аутентификации (в редакторе nano после внесения изменений нажимаем Ctrl+x и на вопрос отвечаем y + enter):
```bash
sudo nano /etc/postgresql/10/main/pg_hba.conf
```

Находим строчку (в конце файла)
```bash
# Database administrative login by Unix domain socket
local   all             postgres                                peer
```

меняем peer на md5
```bash
local   all             postgres                                md5
```

и разрешаем подключения извне (меняем 127.0.0.1/32 на 0.0.0.0/0)
```bash
# IPv4 local connections:
host    all             all             0.0.0.0/0            md5
```

В файле конфигураций разрешаем слушать все IP-адреса
```bash
sudo nano /etc/postgresql/10/main/postgresql.conf
# находим строчку с listen_addresses и меняем значение на
listen_addresses = '*'
```

В конце перезапускаем postgresql
```bash
sudo service postgresql restart
```

3. Заходим под пользователем postgres и добавляем в базу пользователя с правами на чтение
```bash
sudo su -l postgres
psql -U postgres -c "CREATE ROLE readaccess;"
psql -U postgres -c "GRANT SELECT ON ALL TABLES IN SCHEMA public TO readaccess;"
psql -U postgres -c "ALTER DEFAULT PRIVILEGES IN SCHEMA public GRANT SELECT ON TABLES TO readaccess;"
psql -U postgres -c "CREATE USER ds WITH PASSWORD 'ds';"
psql -U postgres -c "GRANT readaccess TO ds;"
```

In [1]:
from psycopg2.extras import DictCursor


In [2]:
import psycopg2

In [29]:
# инициализируем соединение с базой данных
conn = psycopg2.connect(dbname='demo',
                        user='postgres',
                        host='127.0.0.1',
                        port='5432',
                        password='root')

In [4]:
with conn.cursor(cursor_factory=psycopg2.extras.DictCursor) as curr:
    1

In [13]:
# для больших запросов
cur.itersize = 10000

In [19]:
cur.close()

In [22]:
# отправляем запрос


In [24]:
import pandas as pd



In [27]:
# получаем все строки ответа
query = 'SELECT * FROM bookings.aircrafts'
df = pd.read_sql(query, conn)


In [28]:
df

,aircraft_code,model,range
0,773,Boeing 777-300,11100
1,763,Boeing 767-300,7900
2,SU9,Sukhoi SuperJet-100,3000
3,320,Airbus A320-200,5700
4,321,Airbus A321-200,5600
5,319,Airbus A319-100,6700
6,733,Boeing 737-300,4200
7,CN1,Cessna 208 Caravan,1200
8,CR2,Bombardier CRJ-200,2700


In [ ]:
cur.execute('SELECT * FROM ratings order by userid limit 10;')

In [56]:
for row in cur:
    print(row)

['773', 'Boeing 777-300', 11100]
['763', 'Boeing 767-300', 7900]
['SU9', 'Sukhoi SuperJet-100', 3000]
['320', 'Airbus A320-200', 5700]
['321', 'Airbus A321-200', 5600]
['319', 'Airbus A319-100', 6700]
['733', 'Boeing 737-300', 4200]
['CN1', 'Cessna 208 Caravan', 1200]
['CR2', 'Bombardier CRJ-200', 2700]


### Немного про запросы в MySQL

In [ ]:
# для тяжелых запросов к MySQL можно использовать SSDictCursor
import pymysql

connection = pymysql.connect(
    host = 'dbhost',
    port = 3306,
    user = 'dbuser',
    password = 'dbpass',
    db = 'db',
    cursorclass = pymysql.cursors.SSDictCursor
)

with connection.cursor() as cursor:
    cursor.execute(sqlQuery)

    for line in cursor:
        print(line)

### Базы данных и pandas

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_sql('SELECT * FROM ratings limit 10;', conn)
data

## Персональные данные

In [7]:
# плохой вариант для аналитики
database_rows = [
    {'name': 'Натальяgp7', 'emailgp7': 'nata123@mail.rugp7', 'ipgp7': '127.0.0.1gp7'},
    {'name': 'Михаилgp7', 'emailgp7': 'mikha@yandex.rugp7', 'ipgp7': '127.0.0.1gp7'},
]

In [2]:
import hashlib

In [ ]:
hashlib.algorithms_available

In [19]:
m = hashlib.sha256()

m.update(bytes('usernameyandex.ru', encoding="cp1251"))
m.hexdigest()

'c1527fb63ff80c7cbae0ab842c86b3d436571a6a157c2f8c7eb772a363bb3e4b'

In [5]:
m = hashlib.sha256()

m.update(b'usernameyandex.ru')
m.hexdigest()

'c1527fb63ff80c7cbae0ab842c86b3d436571a6a157c2f8c7eb772a363bb3e4b'

### Упражнение
Зашифруйте персональные данные в списке database_rows

In [26]:
import hashlib
m = hashlib.sha256()


In [62]:
database_rows = [
    {'name': 'Натальяgp7', 'emailgp7': 'nata123@mail.rugp7', 'ipgp7': '127.0.0.1gp7'},
    {'name': 'Михаилgp7', 'emailgp7': 'mikha@yandex.rugp7', 'ipgp7': '127.0.0.1gp7'},
]
for el in database_rows:
    for k, v in el.items():
        m = hashlib.sha256()
        m.update(bytes(v, encoding="utf-8"))
        el[k] = m.hexdigest()
database_rows


[{'name': '4b025856b0559a6522d3d24ca712648ccc0fd7943adb0b2615d8ce8e981244e7',
  'emailgp7': 'fa86ed04417d1c9ad40bf13e32d636fc3376d40d5a9fe01b41151ce8701eef30',
  'ipgp7': '753073db78328f2e9ea25e91f2cbaee8dbe1d943b44971a8b31597b9350ef07e'},
 {'name': '80bd4e460f542209cceb7a85736e4643f83994cfad2043b523a8cc933e3ebfd4',
  'emailgp7': 'b245d5be81e7c7377cdf4a9ac8c76d02d04dec27eed29ea1cdd623abb2c93b30',
  'ipgp7': '753073db78328f2e9ea25e91f2cbaee8dbe1d943b44971a8b31597b9350ef07e'}]